In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import os
import sys
import numpy as np
from hls4ml.converters import convert_from_keras_model
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
from callbacks import all_callbacks

os.environ['PATH'] = '/home/bcilab/Xilinx/Vivado/2019.1/bin:' + os.environ['PATH']

In [3]:
DATA_DIR = 'npy'
MODEL_DIR = 'model'

BOARD_NAME = 'ultra96v2'
FPGA_PART = 'xczu3eg-sbva484-1-e'

CLOCK_PERIOD = 10

In [4]:
# Load and scale dataset
#
print("Load and scale dataset")
data = fetch_openml('hls4ml_lhc_jets_hlf')
X, y = data['data'], data['target']
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 5)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)
np.save('y_test.npy', y_test)
np.save('X_test.npy', X_test)
np.save('classes.npy', le.classes_, allow_pickle=True)

Load and scale dataset


In [5]:
# Model
print("Create model")
model = Sequential()
model.add(QDense(64, input_shape=(16,), name='fc1',
                 kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(QActivation(activation=quantized_relu(6), name='relu1'))
model.add(QDense(32, input_shape=(16,), name='fc2',
                 kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(QActivation(activation=quantized_relu(6), name='relu2'))
model.add(QDense(32, input_shape=(16,), name='fc3',
                 kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(QActivation(activation=quantized_relu(6), name='relu3'))
model.add(QDense(5, name='output',
                 kernel_quantizer=quantized_bits(6,0,alpha=1), bias_quantizer=quantized_bits(6,0,alpha=1),
                 kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model.add(Activation(activation='softmax', name='softmax'))

Create model


In [6]:
# Training (or loading model)
#
from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
pruning_params = {'pruning_schedule' : pruning_schedule.ConstantSparsity(0.75, begin_step=2000, frequency=100)}
model = prune.prune_low_magnitude(model, **pruning_params)

/home/bcilab/anaconda3/lib/python3.8/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:212: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  mask = self.add_variable(
/home/bcilab/anaconda3/lib/python3.8/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:219: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  threshold = self.add_variable(
/home/bcilab/anaconda3/lib/python3.8/site-packages/tensorflow_model_optimization/python/core/sparsity/keras/pruning_wrapper.py:233: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self.pruning_step = self.add_variable(


In [8]:
train = True
if train:
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    callbacks = all_callbacks(stop_patience = 1000,
                              lr_factor = 0.5,
                              lr_patience = 10,
                              lr_epsilon = 0.000001,
                              lr_cooldown = 2,
                              lr_minimum = 0.0000001,
                              outputDir = 'model')
    callbacks.callbacks.append(pruning_callbacks.UpdatePruningStep())
    model.fit(X_train_val, y_train_val, batch_size=1024,
              epochs=30, validation_split=0.25, shuffle=True,
              callbacks = callbacks.callbacks)
    # Save the model again but with the pruning 'stripped' to use the regular layer types
    model = strip_pruning(model)
    model.save('model/KERAS_check_best_model.h5')
else:
    from tensorflow.keras.models import load_model
    from qkeras.utils import _add_supported_quantized_objects
    co = {}
    _add_supported_quantized_objects(co)
    model = load_model('model/KERAS_check_best_model.h5', custom_objects=co)

Epoch 1/30
481/487 [============================>.] - ETA: 0s - loss: 0.7399 - accuracy: 0.7514
***callbacks***
saving losses to model/losses.log

Epoch 00001: val_loss improved from inf to 0.74305, saving model to model/KERAS_check_best_model.h5

Epoch 00001: val_loss improved from inf to 0.74305, saving model to model/KERAS_check_best_model_weights.h5

Epoch 00001: saving model to model/KERAS_check_model_last.h5

Epoch 00001: saving model to model/KERAS_check_model_last_weights.h5

***callbacks end***

487/487 [==============================] - 3s 4ms/step - loss: 0.7397 - accuracy: 0.7514 - val_loss: 0.7430 - val_accuracy: 0.7505 - lr: 1.0000e-04
Epoch 2/30
470/487 [===========================>..] - ETA: 0s - loss: 0.7381 - accuracy: 0.7517
***callbacks***
saving losses to model/losses.log

Epoch 00002: val_loss improved from 0.74305 to 0.74053, saving model to model/KERAS_check_best_model.h5

Epoch 00002: val_loss improved from 0.74305 to 0.74053, saving model to model/KERAS_check_

Epoch 13/30
484/487 [============================>.] - ETA: 0s - loss: 0.7212 - accuracy: 0.7559
***callbacks***
saving losses to model/losses.log

Epoch 00013: val_loss did not improve from 0.72677

Epoch 00013: val_loss did not improve from 0.72677

Epoch 00013: saving model to model/KERAS_check_model_last.h5

Epoch 00013: saving model to model/KERAS_check_model_last_weights.h5

***callbacks end***

487/487 [==============================] - 2s 4ms/step - loss: 0.7213 - accuracy: 0.7558 - val_loss: 0.7273 - val_accuracy: 0.7541 - lr: 1.0000e-04
Epoch 14/30
475/487 [============================>.] - ETA: 0s - loss: 0.7206 - accuracy: 0.7559
***callbacks***
saving losses to model/losses.log

Epoch 00014: val_loss improved from 0.72677 to 0.72491, saving model to model/KERAS_check_best_model.h5

Epoch 00014: val_loss improved from 0.72677 to 0.72491, saving model to model/KERAS_check_best_model_weights.h5

Epoch 00014: saving model to model/KERAS_check_model_last.h5

Epoch 00014: saving

Epoch 27/30
473/487 [============================>.] - ETA: 0s - loss: 0.7129 - accuracy: 0.7580
***callbacks***
saving losses to model/losses.log

Epoch 00027: val_loss did not improve from 0.71867

Epoch 00027: val_loss did not improve from 0.71867

Epoch 00027: saving model to model/KERAS_check_model_last.h5

Epoch 00027: saving model to model/KERAS_check_model_last_weights.h5

***callbacks end***

487/487 [==============================] - 2s 4ms/step - loss: 0.7132 - accuracy: 0.7578 - val_loss: 0.7201 - val_accuracy: 0.7570 - lr: 1.0000e-04
Epoch 28/30
487/487 [==============================] - ETA: 0s - loss: 0.7126 - accuracy: 0.7583
***callbacks***
saving losses to model/losses.log

Epoch 00028: val_loss did not improve from 0.71867

Epoch 00028: val_loss did not improve from 0.71867

Epoch 00028: saving model to model/KERAS_check_model_last.h5

Epoch 00028: saving model to model/KERAS_check_model_last_weights.h5

***callbacks end***

487/487 [==============================] -

In [24]:
print("Run prediction")
y_keras = model.predict(X_test)
np.save('y_qkeras.npy', y_keras)

Run prediction


In [25]:
# hls4ml
#
import hls4ml
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

hls_config['Model'] = {}
hls_config['Model']['ReuseFactor'] = 64
hls_config['Model']['Strategy'] = 'Resource'
hls_config['Model']['Precision'] = 'ap_fixed<16,6>'
hls_config['LayerName']['fc1']['ReuseFactor'] = 64
hls_config['LayerName']['fc2']['ReuseFactor'] = 64
hls_config['LayerName']['fc3']['ReuseFactor'] = 64

hls_config['SkipOptimizers'] = ['relu_merge']

hls_model = convert_from_keras_model(model=model,
                                     clock_period=CLOCK_PERIOD,
                                     backend='VivadoAccelerator',
                                     board=BOARD_NAME,
                                     part=FPGA_PART,
                                     io_type='io_stream',
                                     interface='axi_master',
                                     driver='c',
                                     hls_config=hls_config,
                                     output_dir='model')

_ = hls_model.compile()

y_hls = hls_model.predict(np.ascontiguousarray(X_test))

Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: Input
Layer name: fc1, layer type: QDense
Layer name: relu1, layer type: QActivation
Layer name: fc2, layer type: QDense
Layer name: relu2, layer type: QActivation
Layer name: fc3, layer type: QDense
Layer name: relu3, layer type: QActivation
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
Interpreting Sequential
Topology:
Layer name: fc1_input, layer type: InputLayer, input shapes: [[None, 16]], output shape: [None, 16]
Layer name: fc1, layer type: QDense, input shapes: [[None, 16]], output shape: [None, 64]
Layer name: relu1, layer type: Activation, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: fc2, layer type: QDense, input shapes: [[None, 16]], output shape: [None, 32]
Layer name: relu2, layer type: Activation, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: fc3, layer type: QDense, input shapes: [[None, 16]], output shape: [None, 32]
Layer nam

In [26]:
from sklearn.metrics import accuracy_score
print('-----------------------------------')
print('Keras  Accuracy: {}'.format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print('hls4ml Accuracy: {}'.format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))
print('-----------------------------------')

-----------------------------------
Keras  Accuracy: 0.7562831325301205
hls4ml Accuracy: 0.7560361445783133
-----------------------------------


In [27]:
hls_model.build(csim=False, synth=True, export=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.1 (64-bit)
  **** SW Build 2552052 on Fri May 24 14:47:09 MDT 2019
  **** IP Build 2548770 on Fri May 24 18:01:18 MDT 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /home/bcilab/Xilinx/Vivado/2019.1/scripts/vivado_hls/hls.tcl -notrace
INFO: Applying HLS Y2K22 patch v1.2 for IP revision
INFO: [HLS 200-10] Running '/home/bcilab/Xilinx/Vivado/2019.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'bcilab' on host 'bcilab-rrid411' (Linux_x86_64 version 4.15.0-189-generic) on Wed Jul 27 13:17:16 IST 2022
INFO: [HLS 200-10] On os Ubuntu 18.04.6 LTS
INFO: [HLS 200-10] In directory '/home/bcilab/NN_fpga/guissepe_notebook/model'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Opening project '/home/bcilab/NN_fpga/guissepe_notebook/model/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject_axi.cpp' to the project
INFO: [HLS 200-10] Adding design file 'f

INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' into 'nnet::dense_wrapper<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_dense_resource.h:99->firmware/nnet_utils/nnet_dense_resource.h:524->firmware/nnet_utils/nnet_dense_stream.h:24) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' into 'nnet::dense_wrapper<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' (firmware/nnet_utils/nnet_dense_resource.h:99->firmware/nnet_utils/nnet_dense_resource.h:524->firmware/nnet_utils/nnet_dense_stream.h:24) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6

INFO: [HLS 200-489] Unrolling loop 'ReLUPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:67) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, 64u>, nnet::array<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 64u>, relu_config4>' completely with a factor of 64.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_types.h:32) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, nnet::array<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, relu_config7>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'ReLUPackLoop' (firmware/nnet_utils/nnet_activation_stream.h:67) in function 'nnet::relu<nnet::array<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, nnet::array<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, 32u>, relu_config7>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_types.h:32) in function 'nnet::relu<nne

INFO: [HLS 200-489] Unrolling loop 'InitAccum' (firmware/nnet_utils/nnet_dense_resource.h:58) in function 'nnet::dense_wrapper<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:73) in function 'nnet::dense_wrapper<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' completely with a factor of 1024.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_resource.h:97) in function 'nnet::dense_wrapper<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'InitAccum' (firmware/nnet_utils/nnet_dense_resource.h:58) in function 'nnet::dense_wrapper<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (a

INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' into 'nnet::dense_wrapper<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (firmware/nnet_utils/nnet_dense_resource.h:99->firmware/nnet_utils/nnet_dense_resource.h:524->firmware/nnet_utils/nnet_dense_stream.h:24) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' into 'nnet::dense_wrapper<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config5>' (firmware/nnet_utils/nnet_dense_resource.h:99->firmware/nnet_utils/nnet_dense_resource.h:524->firmware/nnet_utils/nnet_dense_stream.h:24) automatically.
INFO: [XFORM 203-602] Inlining function 'nnet::cast<ap_ufixed<6, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6

...

INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.93 seconds; current allocated memory: 620.554 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'linear_array_array_ap_fixed_32u_linear_config6_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'linear<array,array<ap_fixed,32u>,linear_config6>'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 1.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 2.32 seconds; current allocated memory: 623.613 MB.
INFO: [BIND 205-100] Starting micro-

INFO: [SCHED 204-61] Pipelining function 'softmax_latency<array,array,softmax_config13>'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 3.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 0.12 seconds; current allocated memory: 672.988 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Starting global binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.2 seconds; current allocated memory: 673.461 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'softmax_array_array_ap_fixed_5u_softmax_config13_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting sche

INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'linear_array_array_ap_fixed_32u_linear_config9_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'linear_array_array_ap_fixed_32u_linear_config9_s'.
INFO: [HLS 200-111]  Elapsed time: 3.46 seconds; current allocated memory: 1.103 GB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'relu_array_array_ap_ufixed_32u_relu_config10_s' 
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'relu_array_array_ap_ufixed_32u_relu_config10_s'.
INFO: [HLS 200-111]  Elapsed time: 2.44 seconds; current allocated memory: 1.110 GB.
INFO: [HLS 200-10] ----------------------------------------------------------------
IN

INFO: [RTMG 210-285] Implementing FIFO 'start_for_linear_array_array_ap_fixed_5u_linear_config12_U0_U(start_for_linear_array_array_ap_fixed_5u_linear_config12_U0)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'start_for_softmax_array_array_ap_fixed_5u_softmax_config1g8j_U(start_for_softmax_array_array_ap_fixed_5u_softmax_config1g8j)' using Shift Registers.
INFO: [RTMG 210-278] Implementing memory 'myproject_axi_tmp_data_V_ram (RAM)' using block RAMs.
INFO: [RTMG 210-285] Implementing FIFO 'in_local_V_data_0_V_U(fifo_w16_d16_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'in_local_V_data_1_V_U(fifo_w16_d16_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'in_local_V_data_2_V_U(fifo_w16_d16_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'in_local_V_data_3_V_U(fifo_w16_d16_A)' using Shift Registers.
INFO: [RTMG 210-285] Implementing FIFO 'in_local_V_data_4_V_U(fifo_w16_d16_A)' using Shift Registers.
INFO: [RTMG 210-28

{'EstimatedClockPeriod': '8.750',
 'BestLatency': '123',
 'WorstLatency': '123',
 'IntervalMin': '124',
 'IntervalMax': '124',
 'BRAM_18K': '352',
 'DSP48E': '515',
 'FF': '14051',
 'LUT': '120463',
 'URAM': '0',
 'AvailableBRAM_18K': '432',
 'AvailableDSP48E': '360',
 'AvailableFF': '141120',
 'AvailableLUT': '70560',
 'AvailableURAM': '0'}

In [28]:
 hls4ml.writer.vivado_accelerator_writer.VivadoAcceleratorWriter.write_header_file(hls_model, X_test, y_test, y_keras, y_hls, 64, 'model/sdk/common/data.h')

In [29]:
hls4ml.report.read_vivado_report('model/')

Found 1 solution(s) in model//myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject_axi'
* Date:           Wed Jul 27 13:26:13 2022

* Version:        2019.1 (Build 2552052 on Fri May 24 15:28:33 MDT 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: zynquplus
* Target device:  xczu3eg-sbva484-1-e


== Performance Estimates
+ Timing (ns): 
    * Summary: 
    +--------+-------+----------+------------+
    |  Clock | Target| Estimated| Uncertainty|
    +--------+-------+----------+------------+
    |ap_clk  |  10.00|     8.750|        1.25|
    +--------+-------+----------+------------+

+ Latency (clock cycles): 
    * Summary: 
    +-----+-----+-----+-----+---------+
    |  Latency  |  Interval | Pipeline|
    | min | max | min | max |   Type  |
    +-----+-----+-----+-----+---------+
    |  123|  123|  123|  123|   none  |
    +-----+-----+-----+-----+---------+

    + De

In [ ]:
# Generate bitstream and HDF file
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)